# MLFlow Basics

## O que você irá aprender aqui?

Este notebook utiliza um dataset didático para aprender os conceitos básicos de MLFlow:
* Criar um experimento de Machine Learning;
* Rastrear parâmetros e métricas das execuções do experimento;
* Configurar a interface do MLFlow
* Dispor o modelo em uma REST API utilizando um dos modelos do experimento.

## Sobre o Dataset

Aqui utilizaremos um dataset bem conhecido da comunidade de Ciência de Dados, de modo que possamos focar diretamente na execução de experimentos.

O Dataset contém as seguintes colunas:

- SepalLengthCm
- SepalWidthCm
- pPetalLengthCm
- PetalWidthCm
- Species: ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

O propósito é classificar as espécies de flores do gênero Iris dado suas características.

## Importação de bibliotecas e carregamento do Dataset

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from io import BytesIO
from PIL import Image
import mlflow
import mlflow.sklearn
import warnings

warnings.filterwarnings("ignore")

In [58]:
# Carregar o dataset
# Carregar o dataset
df = pd.read_csv('./data/iris.csv', index_col='Id')

df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [59]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## Preparando os dados para a modelagem

Como o propósito é didático, não faremos nenhum estudo analítico avançado nos dados. Vamos aplicar as técnicas básicas de ML para variáveis categóricas e numéricas e depois aplicaremos alguns modelos com diversos parâmetros e comparar os resultados utilizando as métricas convencionais de modelos de classificação.

In [50]:
X = df.drop(columns=['Species'])
y = df['Species']

# Dividir o dataset em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Pré-processamento: StandardScaler para variáveis numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.columns)
    ]
)

In [54]:
# Criando objeto com as configurações do modelo
# model_config = {
#     'model': RandomForestClassifier,
#     'params': {
#         'random_state': 42,
#         'n_estimators': 10,
#         'max_depth': 3
#     }
# }

model_config = {
    'model': LogisticRegression,
    'params': {
        'random_state': 42,
        'C': 0.3,
        'penalty': 'l2'
    }
}

In [55]:
def get_confusion_matrix_img(y_test, y_pred, df):
    # Gere a matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Crie a figura da matriz de confusão
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=df['Species'].unique())
    disp.plot()

    # Salve a figura em um objeto BytesIO
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()

    # Converta o buffer para uma imagem PIL
    return Image.open(buf)

In [56]:
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor), 
        ('model', model_config['model'](**model_config['params']))
    ]
)

# Vamos dar um nome para as execuções do modelo para podermos localizar futuramente na interface do MLFlow
mlflow.set_experiment('iris-classification')

with mlflow.start_run():
    model = pipeline.fit(X_train, y_train)
    
    # Fazer previsões em treino e teste
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    # Calcular métricas de teste
    accuracy_train = accuracy_score(y_train, y_pred_train)
    precision_train = precision_score(y_train, y_pred_train,  average='macro')
    recall_train = recall_score(y_train, y_pred_train,  average='macro')
    f1_train = f1_score(y_train, y_pred_train,  average='macro')
    
    # Calcular métricas de teste
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test,  average='macro')
    recall_test = recall_score(y_test, y_pred_test,  average='macro')
    f1_test = f1_score(y_test, y_pred_test,  average='macro')

    # Registrar parâmetros e métricas no MLFlow
    mlflow.log_param('model', model_config['model'].__name__)
    mlflow.log_params(model_config['params'])
    
    mlflow.log_metric('accuracy_test', accuracy_test)
    mlflow.log_metric('accuracy_train', accuracy_train)
    
    mlflow.log_metric('precision_test', precision_test)
    mlflow.log_metric('precision_train', precision_train)

    mlflow.log_metric('recall_test', recall_test)
    mlflow.log_metric('recall_train', recall_train)

    mlflow.log_metric('f1_score_test', f1_test)
    mlflow.log_metric('f1_score_train', f1_train)

    mlflow.log_image(get_confusion_matrix_img(y_test, y_pred_test, df), "confusion_matrix.png")
    
    # Registrar o modelo no MLFlow
    mlflow.sklearn.log_model(model, 'model')

# Imprimir resultados
print(f"Modelo: {model_config['model'].__name__}")
print(f"Parâmetros: {model_config['params']}")
print("Resultados do teste:")
print(f"Acurácia: {accuracy_test}")
print(f"Precisão: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")


Modelo: LogisticRegression
Parâmetros: {'random_state': 42, 'C': 0.3, 'penalty': 'l2'}
Resultados do teste:
Acurácia: 0.9777777777777777
Precisão: 0.9761904761904763
Recall: 0.9743589743589745
F1 Score: 0.974320987654321


## Visualizando os resultados na interface do MLFlow

Basta rodar no terminal o comando:

`mlflow ui --port 8100`

Lembrando que o comando deve ser executado na mesma pasta em que está o arquivo Jupyter Notebook ou .py que você executou o treinamento do modelo.

Agora você pode alterar o modelo e/ou os parâmetros do modelo na célula anterior para criar novas runs e monitorar depois na interface do MLFlow.

## Inserindo o modelo em uma REST API

Para servir um modelo basta coletar o ID de uma run de um exeprimento e executar no terminal:
 
`mlflow models serve -m "runs:/<run-id>/model" -p 1234`

Para testar, uitizamos a lib requests passando as informações de um novo diamante.

In [61]:

import requests

data = {"dataframe_records": [{"SepalLengthCm": 5.1, "SepalWidthCm": 3.5, "PetalLengthCm": 1.4, "PetalWidthCm": 0.2}]}

requests.post("http://localhost:1234/invocations", json=data).json()

{'predictions': ['Iris-setosa']}